# Covid-19 Data Science Project

Eine Datenanalyse zu Covid-19 Daten. Ziel dieser Datenanalyse ist es ein Modell zu entwickeln, dass die zukünftigen Fallzahlen anhand verschiedener Input-Variablen wie dem Standort, den aktuellen Maßnahmen(Lockdown) und den vorherigen Fallzahlen vorhersagen kann. Zur Analyse werden Regressionsmodelle verwendet. Anschließend wird ein Deep Learning Algorithmus(neuronales Netz) angewendet.